# Expectation Maximisation coin-toss. - WIP

Illustration of the expectation-maximisation (EM) algorithm on a coin-toss problem. Based on Nature's ["What is the expectation maximization algorithm?"](http://ai.stanford.edu/~chuongdo/papers/em_tutorial.pdf).

consider a coin-flipping experiment in which we are given a pair
of coins $A$ and $B$ of unknown biases. On any given flip, coin $A$ will land on heads with probability $\theta_A$ and tails with probability $1 - \theta_A$ and similarly for coin $B$ which has bias $\theta_B$.

Lets say that we run an experiment where we repeat the following procedure five times:

* Randomly choose one of the two coins out of a bag (we don't know which coin we take, each coin can be taken with equal probablity).
* Perform $m$ independent coin tosses with the selected coin and count how many times heads is observed.
* Put the selected coin back in the bag.
* Repeat this $n$ times.

Our goal now is to estimate $\theta_A$ and $\theta_B$ from this experiment. The problem is that we have incomplete data, we don't know which coin we picked, the selection of coin $A$ or $B$ is a latent variable.

This notebook will illustrate how we can estimate $\theta_A$ and $\theta_B$ with help of the EM algorithm. We will refer to both of them together as the model parameters $\theta = \{\theta_A, \theta_B\}$.

* https://am207.github.io/2017/wiki/EM.html
* https://stats.stackexchange.com/questions/72774/numerical-example-to-understand-expectation-maximization
* https://math.stackexchange.com/questions/25111/how-does-expectation-maximization-work
* https://math.stackexchange.com/questions/81004/how-does-expectation-maximization-work-in-coin-flipping-problem
* https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html
* https://am207.github.io/2017/wiki/EM.html
* https://media.nature.com/original/nature-assets/nbt/journal/v26/n8/extref/nbt1406-S1.pdf
* https://mk-minchul.github.io/EM/
* log-likelihood:
    * https://blog.metaflow.fr/ml-notes-why-the-log-likelihood-24f7b6c40f83
    * https://stats.stackexchange.com/questions/174481/why-to-optimize-max-log-probability-instead-of-probability
* https://media.nature.com/original/nature-assets/nbt/journal/v26/n8/extref/nbt1406-S1.pdf
* http://www.rmki.kfki.hu/~banmi/elte/bishop_em.pdf

In [1]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

sns.set_style('darkgrid')
%matplotlib inline
warnings.filterwarnings('ignore')
np.random.seed(42)

Each experiment will have observations the number of times heads was seen for each coin picked. Lets represent this as $X = (x_1, ... , x_n)$ for $n$ the number of times a coin was chosen from the bag. $x_i$ is the number of times heads was observed for the $i$-th picked coin.

Note that for each coin $i$ drawn we don't actually know the true identity ($A$ or $B$). We will represent this unknown as the latent variable $Z = (z_i, ... , z_n)$ with $z_i \in \{A, B\}$.

Note that this means that $x_i$ is dependent on $z_i$.

![Latent variable graph](https://i.imgur.com/jphWJbX.png)

In [2]:
m = 10  # Number of flips per coin drawn
n = 50  # Number of times coin is drawn from bag
theta_A = 0.8  # Bias of coin A towards landing on heads
theta_B = 0.3  # Bias of coin B towards landing on heads
theta = (theta_A, theta_B)
p_A = 0.5  # Probability of picking A
p_B = 0.5  # Probability of picking B
p = (p_A, p_B)

# Choose between A or B (note that is is latent and not known when we run EM)
# Note that z should be either A or B but we'll represent those by there bias here
zs = np.random.choice(theta, size=n, p=p, replace=True)
print('zs: ', zs)

# Observed number of head for each coin drawn
xs = np.random.binomial(n=m, p=zs)
print('xs: ', xs)

zs:  [ 0.8  0.3  0.3  0.3  0.8  0.8  0.8  0.3  0.3  0.3  0.8  0.3  0.3  0.8  0.8
  0.8  0.8  0.3  0.8  0.8  0.3  0.8  0.8  0.8  0.8  0.3  0.8  0.3  0.3  0.8
  0.3  0.8  0.8  0.3  0.3  0.3  0.8  0.8  0.3  0.8  0.8  0.8  0.8  0.3  0.8
  0.3  0.8  0.3  0.3  0.8]
xs:  [ 5  4  5  5  8  6 10  2  1  2  8  2  4  9  9  8  9  4 10  5  4  9 10  7  7
  4  7  1  2  9  5  8  9  1  2  2  7  8  5  8  9  7  7  3  7  3  8  3  0  9]


The [expectation-maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm)  will help us estimate the values for latent variables $\theta_A$ and $\theta_B$. It will do this by trying to find the maximum likelihood of the parameters $\theta$ in the latent-variable model in an iterative way. The algorithm starts with an initial guess of the parameters $\theta_A$ and $\theta_B$ and then follows an alternating 2-step procedure until convergence:

1. **E-step**: Given the current guess of $\theta_A$ and $\theta_B$ and the observations $X$, determine for each $x_i$ how likely each parameter ($\theta_A$ or $\theta_B$) was to have generated the sequence of observed heads. We can then assign weights to each sample according to how likely it was generated by coin $A$ or $B$.

2. **M-step**: Given the weighted data from the previous step, update the guesses for $\theta_A$ and $\theta_B$ by maximizing their likelihoods.


![EM algorithm](https://i.imgur.com/Taa1gcv.png)


The [expectation-maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) will try to optimize the likelihood of the dataset $X$ given the parameters $\theta$. We'll write this as:

$$ \max_{\theta} p(X \mid \theta) = \max_{\theta} \prod_{i=1}^N p(x_i \mid \theta) = \max_{\theta} \prod_{i=1}^N \sum_c^{\{A,B\}} p(x_i, z_i=c \mid \theta)$$



The expectation maximization algorithm is a refinement on this basic idea. Rather than picking the single most likely completion of the missing coin assignments on each iteration, the expectation maximization algorithm computes probabilities for each possible completion of the missing data, using the current parameters θˆ(t). These probabilities are used to create a weighted training set consisting of all possible completions of the data. Finally, a modified version of maximum likelihood estimation that deals with weighted training examples provides new parameter estimates, θ ˆ(t+1). By using weighted training examples rather than choosing the single best completion, the expectation maximization algorithm accounts for the confidence of the model in each completion of the data


In summary, the expectation maximization algorithm alternates between the steps of guessing a probability distribution over completions of missing data given the current model (known as the E-step) and then reestimating the model parameters using these completions (known as the M-step).


The name ‘E-step’ comes from the fact that one does not usually need to form the probability distribution over completions explicitly, but rather need only compute ‘expected’ sufficient statistics over these completions

the name ‘M-step’ comes from the fact that model reestimation can be thought of as ‘maximization’ of the expected log-likelihood of the data




More pragmatically speaking, the EM algorithm is an iterative method that alternates between computing a conditional expectation and solving a maximization problem, hence the name expectation maximization


We do not know z, so instead of using the true values of z we're going to use their expectations. In particular we wil compute \theta by maximizing its likelihood under the expected values of z.

## E-step (Expectation)



In the E-step, a [likelihood function](https://en.wikipedia.org/wiki/Likelihood_function) of latent variables $z_i$ given the data $x_i$ is computed using the current estimates of $\theta$. This likelihood function describes the plausibility of the latent variables $z_i$ being assigned to $A$ or $B$. For each $z_i$ we can write this as:

$$ p(z_i=c \mid x_i, \theta) \quad c \in \{A, B\}$$

The

Since $\theta$ is fixed and simplifying the notation we write this as $p(c \mid x_i)$. We can then use the [Bayes rule](https://en.wikipedia.org/wiki/Bayes%27_theorem ) to write:

$$ p(c \mid x_i) = \frac{p(x_i \mid c) p(c)}{p(x_i)} $$

Now $p(c)=0.5$ since we pick one out of 2 coins out of the bag. 

$p(x_i \mid c)$ is the probabiliy of $x_i$ times head out of $m$ flips when coin $c$ was drawn. It dependents only on $\theta_c$ and follows the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) by definition ($m$ coinflips with $x_i$ heads).

$$p(x_i \mid c) = {m \choose x_i}\,\theta_c^{x_i}(1-\theta_c)^{m-x_i}$$


$p(x_i)$ is the distribution of observing the number of heads in the sequence without knowing which coin is selected and is equal to $p(x_i \mid A)p(A) + p(x_i \mid B)p(B)$.

Now, since the [binomial coeffient](https://en.wikipedia.org/wiki/Binomial_coefficient) appears in all terms of our fraction we can remove it and write the likelihood function as:

$$ p(c \mid x_i) = \frac{ \theta_c^{x_i}(1-\theta_c)^{m-x_i} p(c)}{\theta_A^{x_i}(1-\theta_A)^{m-x_i} p(A) + \theta_B^{x_i}(1-\theta_B)^{m-x_i} p(B)}  $$


### Example

Lets say we start out with initial parameter guesses $\theta_A = 0.6$ and $\theta_B = 0.5$, we pick one coin at random ($p=0.5$) and observe 8 times heads and 2 times tails. If we will in the equations we find that:

$$
\theta_A^{x_i}(1-\theta_A)^{m - x_i} p(A) = 0.6^8*(1-0.6)^2*0.5 ≈ 0.0013 \\
\theta_B^{x_i}(1-\theta_B)^{m - x_i} p(B) = 0.5^8*(1-0.5)^2*0.5 ≈ 0.0004 \\
\Downarrow \\
p(A \mid x_i=8) = \frac{0.0013}{0.0013 + 0.0004} ≈ 0.73\\
p(B \mid x_i=8) = \frac{0.0004}{0.0013 + 0.0004} ≈ 0.27
$$

In [3]:
# E-step: calculate probability distributions over possible completions

def e_step(tA, tB):
    # Weighted data over n chosen coins
    w_A_H = 0
    w_A_T = 0
    w_B_H = 0
    w_B_T = 0

    # For each chosen coin
    for x in xs:
        coef_A = ((tA**x) * ((1-tA)**(m-x))) * p_A
        coef_B = ((tB**x) * ((1-tB)**(m-x))) * p_B

        coef_AB = coef_A + coef_B

        p_A_x = coef_A / coef_AB
        p_B_x = coef_B / coef_AB

        w_A_H += p_A_x * x
        w_A_T += p_A_x * (m - x)
        w_B_H += p_B_x * x
        w_B_T += p_B_x * (m - x)
    return w_A_H, w_A_T, w_B_H, w_B_T

In [4]:
# M -step
def m_step(w_A_H, w_A_T, w_B_H, w_B_T):
    tA = w_A_H / (w_A_H + w_A_T)
    tB = w_B_H / (w_B_H + w_B_T)
    return tA, tB

In [5]:
tA, tB = 0.51, 0.5

for i in range(10):
    print('Step {}: A={}, B={}'.format(i, tA, tB))
    w_A_H, w_A_T, w_B_H, w_B_T = e_step(tA, tB)
    tA, tB = m_step(w_A_H, w_A_T, w_B_H, w_B_T)

Step 0: A=0.51, B=0.5
Step 1: A=0.5902116444208838, B=0.5573355860971709
Step 2: A=0.6281270259605728, B=0.5197970793429936
Step 3: A=0.7224739053517539, B=0.421905516120847
Step 4: A=0.7973640474115157, B=0.33034231342770287
Step 5: A=0.8097749133059613, B=0.3138660059289479
Step 6: A=0.8108856254648229, B=0.3126081748766472
Step 7: A=0.811010957899099, B=0.3125615147577201
Step 8: A=0.8110385124408093, B=0.3125781746882331
Step 9: A=0.811048367275363, B=0.31258799858636593
